In [1]:
import mysql.connector
import getpass

print("1. Open mySQL workbench, local instance, run createAll.sql and loadAll.sql")
print("2. Login as root and establish connection with client")

db_user = getpass.getpass('Enter username') #root 
db_password = getpass.getpass('Enter password') #taishiseiga9
config = {
   'user': db_user,
   'password': db_password,
   'host': 'localhost',
   'port': 3306,
   'database': 'wonderland',
   'raise_on_warnings': True                    
}
db_connection = mysql.connector.connect(**config)

print("3. Run activities and queries ")

1. Open mySQL workbench, local instance, run createAll.sql and loadAll.sql
2. Login as root and establish connection with client


Enter username ········
Enter password ········


3. Run activities and queries 


In [7]:
print("A.1 Clerk Creating a New Ballot")
def create_new_ballot(db_connection):
    cursor = db_connection.cursor()
    ballot_id = input("Enter the Ballot ID: ")
    option = input("Enter option: ")
    availability_start = input("Enter the start time (YYYY-MM-DD HH:MM:SS): ")
    availability_end = input("Enter the end time (YYYY-MM-DD HH:MM:SS): ")
    for option in ['YES', 'NO', 'ABSTAIN']:
        query = f"""
        INSERT INTO Ballots (ballot_id, question_text, availability_time_start, availability_time_end)
        VALUES ('{ballot_id}', '{option}', '{availability_start}', '{availability_end}');
        """
        cursor.execute(query)
    db_connection.commit()
    print(f"Ballot {ballot_id} created successfully!")

print("4. Example Ballot, copy into input: B100, ABSTAIN, 2024-11-01 00:00:00, 2024-11-04 23:59:59")
create_new_ballot(db_connection)

A.1 Clerk Creating a New Ballot
4. Example Ballot, copy into input: B100, ABSTAIN, 2024-11-01 00:00:00, 2024-11-04 23:59:59


Enter the Ballot ID:  B100
Enter option:  ABSTAIN
Enter the start time (YYYY-MM-DD HH:MM:SS):  2024-11-01 00:00:00
Enter the end time (YYYY-MM-DD HH:MM:SS):  2024-11-04 23:59:59


Ballot B100 created successfully!


In [25]:
from mysql.connector import Error 
print("A.2 Folk Registering to Vote")
def register_to_vote(connection):
    cursor = connection.cursor()
    person_id = input("Enter your Person ID: ")
    voting_center = input("Enter the acronym of the voting center: ")
    vote_time = input("Enter the vote time (YYYY-MM-DD HH:MM:SS): ")
    query = f"""
    INSERT INTO VotingRegistry (registry_id, person_id, acronym, vote_time)
    VALUES ('R100', '{person_id}', '{voting_center}', '{vote_time}');
    """
    try:
        cursor.execute(query)
        connection.commit()
        print("Successfully registered to vote!")
    except Error as e:
        print(f"Registration failed: {e}")

print("5. 0000000000000001, VC01, 2024-11-01 12:00:00")
register_to_vote(db_connection)

A.2 Folk Registering to Vote
5. 0000000000000001, VC01, 2024-11-01 12:00:00


Enter your Person ID:  0000000000000001
Enter the acronym of the voting center:  VC01
Enter the vote time (YYYY-MM-DD HH:MM:SS):  2024-11-01 12:00:00


Registration failed: 1644 (45000): Registration date to vote in VotingRegistry does not fall within the operating periods of a VotingCenter


In [19]:
print("A.3 Clerk Modifying the Availability Period")
def modify_ballot_period(connection):
    cursor = connection.cursor()
    ballot_id = input("Enter the Ballot ID to modify: ")
    new_start = input("Enter the new start time (YYYY-MM-DD HH:MM:SS): ")
    new_end = input("Enter the new end time (YYYY-MM-DD HH:MM:SS): ")
    query = f"""
    UPDATE Ballots
    SET availability_time_start = '{new_start}', availability_time_end = '{new_end}'
    WHERE ballot_id = '{ballot_id}';
    """
    try:
        cursor.execute(query)
        connection.commit()
        print(f"Ballot {ballot_id} availability updated successfully!")
    except Error as e:
        print(f"Failed to modify ballot period: {e}")

print("6. B001, 2024-11-01 00:00:01, 2024-11-04 23:59:59")
modify_ballot_period(db_connection)

A.3 Clerk Modifying the Availability Period
6. B001, 2024-11-01 00:00:01, 2024-11-04 23:59:59


Enter the Ballot ID to modify:  B001
Enter the new start time (YYYY-MM-DD HH:MM:SS):  2024-11-01 00:00:01
Enter the new end time (YYYY-MM-DD HH:MM:SS):  2024-11-04 23:59:59


Ballot B001 availability updated successfully!


In [29]:
print("A.4 Voter Casting a Ballot")
from mysql.connector import Error

def cast_vote(connection):

    # Get user input
    person_id = input("Enter your Person ID: ")
    ballot_id = input("Enter the Ballot ID: ")
    vote_choice = input("Enter your vote (YES, NO, ABSTAIN): ")
    vote_time = input("Enter the vote time (YYYY-MM-DD HH:MM:SS): ")

    try:
        # Start a transaction
        connection.start_transaction()
        cursor = connection.cursor()

        # Step 1: Validate registration
        query_validate = """
        SELECT registry_id 
        FROM VotingRegistry
        WHERE person_id = %s 
          AND vote_time <= %s;
        """
        cursor.execute(query_validate, (person_id, vote_time))
        registration = cursor.fetchone()

        if not registration:
            raise Exception("Invalid registration: No valid registration found for this person and vote time.")

        registry_id = registration[0]

        # Step 2: Check for duplicate vote
        query_duplicate_check = """
        SELECT COUNT(*)
        FROM CastedVote
        WHERE registry_id = %s
          AND ballot_id = %s;
        """
        cursor.execute(query_duplicate_check, (registry_id, ballot_id))
        duplicate_count = cursor.fetchone()[0]

        if duplicate_count > 0:
            raise Exception("Duplicate vote detected: A person can cast only one vote per ballot.")

        # Step 3: Cast the vote
        query_cast_vote = """
        INSERT INTO CastedVote (vote_id, registry_id, ballot_id, choice, cast_time)
        VALUES (UUID(), %s, %s, %s, %s);
        """
        cursor.execute(query_cast_vote, (registry_id, ballot_id, vote_choice, vote_time))

        # Commit the transaction if everything is successful
        connection.commit()
        print("Vote successfully cast!")

    except Exception as e:
        # Rollback in case of error
        connection.rollback()
        print(f"Failed to cast vote: {e}")
    finally:
        if cursor:
            cursor.close()

print("7. 0000000000000001, B001, ABSTAIN, 2024-11-01 09:00:01")
cast_vote(db_connection)

A.4 Voter Casting a Ballot
7. 0000000000000001, B001, ABSTAIN, 2024-11-01 09:00:01


Enter your Person ID:  0000000000000001
Enter the Ballot ID:  B001
Enter your vote (YES, NO, ABSTAIN):  ABSTAIN
Enter the vote time (YYYY-MM-DD HH:MM:SS):  2024-11-01 09:00:01


Failed to cast vote: Invalid registration: No valid registration found for this person and vote time.


In [23]:
print("A.5 Staff Removing a Folk")
def remove_folk(connection):
    cursor = connection.cursor()
    person_id = input("Enter the Person ID to remove: ")
    query = f"DELETE FROM Folks WHERE person_id = '{person_id}';"
    try:
        cursor.execute(query)
        connection.commit()
        print(f"Person {person_id} and all associated data removed successfully.")
    except Error as e:
        print(f"Failed to remove person: {e}")
print("8. Remove '0000000000000001'")
remove_folk(db_connection)

A.5 Staff Removing a Folk
8. Remove '0000000000000001'


Enter the Person ID to remove:  0000000000000001


Person 0000000000000001 and all associated data removed successfully.


In [ ]:
print("Query 1")
# Define the query
query = """
SELECT F.first_name, F.last_name, F.nickname, F.residence_city, FE.email_address 
FROM Folks F 
LEFT JOIN Folks_email FE ON F.person_id = FE.id;
"""

# Execute the query
cur = db_connection.cursor()
cur.execute(query)

# Print header
print(f"{'First Name':15} {'Last Name':15} {'Nickname':15} {'Residence City':20} {'Email Address':30}")
print("=" * 95)

# Iterate over the result set and handle NULLs
for row in cur:
    first_name, last_name, nickname, residence_city, email_address = row
    print(f"{(first_name or ''):15} {(last_name or ''):15} {(nickname or ''):15} {(residence_city or ''):20} {(email_address or ''):30}")

# Close the cursor to clear any remaining results
cur.close()


In [ ]:
print("Query 2")
cur = db_connection.cursor()
query = """
SELECT residence_city AS city, residence_state AS state, COUNT(*) AS num_residents
FROM Folks
GROUP BY residence_city, residence_state
HAVING COUNT(*) > 0
ORDER BY num_residents DESC;
"""
cur.execute(query)
print(f"{'City':20} {'State':15} {'Num Residents':10}")
print("=" * 50)
for row in cur:
    print(f"{row[0]:20} {row[1]:15} {row[2]:10}")
cur.close()


In [ ]:
print("Query 3")
cur = db_connection.cursor()
query = """
SELECT VC.street_number, VC.street_name, VC.city, VC.state, VC.zipcode, 
       COUNT(VR.registry_id) AS num_registered
FROM VotingCenter VC
LEFT JOIN VotingRegistry VR 
    ON VC.street_number = VR.street_number 
   AND VC.street_name = VR.street_name 
   AND VC.city = VR.city 
   AND VC.state = VR.state 
   AND VC.zipcode = VR.zipcode
GROUP BY VC.street_number, VC.street_name, VC.city, VC.state, VC.zipcode
ORDER BY VC.zipcode;
"""
cur.execute(query)
print(f"{'Street Number':15} {'Street Name':20} {'City':15} {'State':10} {'Zipcode':10} {'Num Registered':10}")
print("=" * 80)
for row in cur:
    print(f"{row[0]:15} {row[1]:20} {row[2]:15} {row[3]:10} {row[4]:10} {row[5]:10}")
cur.close()


In [ ]:
print("Query 4")
cur = db_connection.cursor()
cur.execute("SET @center_acronym = 'VC01';")
cur.execute("SET @start_date = '2024-11-01';")
cur.execute("SET @end_date = '2024-11-04';")
query = """
SELECT DISTINCT F.person_id, F.first_name, F.last_name, F.nickname
FROM Folks F
JOIN VotingRegistry VR ON F.person_id = VR.person_id
WHERE VR.acronym = @center_acronym
  AND VR.vote_time BETWEEN @start_date AND @end_date;
"""
cur.execute(query)
print(f"{'Person ID':20} {'First Name':15} {'Last Name':15} {'Nickname':15}")
print("=" * 65)
for row in cur:
    print(f"{row[0]:20} {row[1]:15} {row[2]:15} {row[3]:15}")
cur.close()


In [ ]:
print("Query 5")
cur = db_connection.cursor()
cur.execute("SET @month = '2024-11';")
cur.execute("SET @excluded_centers = '\"VC01\", \"VC03\"';")
cur.execute("SET @megapolis_x = 0;")
cur.execute("SET @megapolis_y = 0;")
query = """
SELECT COUNT(DISTINCT VR.registry_id) AS num_unique_registrations
FROM VotingRegistry VR
JOIN Places P ON VR.street_number = P.street_number 
   AND VR.street_name = P.street_name 
   AND VR.city = P.city 
   AND VR.state = P.state 
   AND VR.zipcode = P.zipcode
WHERE P.coordinates IS NOT NULL
  AND (
      POW(SUBSTRING_INDEX(P.coordinates, ',', 1) - @megapolis_x, 2) + 
      POW(SUBSTRING_INDEX(P.coordinates, ',', -1) - @megapolis_y, 2)
  ) <= POW(5, 2)
  AND DATE_FORMAT(VR.vote_time, '%Y-%m') = @month
  AND VR.acronym NOT IN ('VC02', 'VC03');
"""
cur.execute(query)
result = cur.fetchone()
print(f"Number of unique registrations: {result[0]}")
cur.close()


In [ ]:
print("Query 6")
cur = db_connection.cursor()
cur.execute("SET @city = 'City1';")
cur.execute("SET @start_date = '2024-11-01';")
cur.execute("SET @end_date = '2024-11-30';")
query = """
SELECT VR.acronym, COUNT(*) AS total_registrations
FROM VotingRegistry VR
INNER JOIN Places P USING (street_number, street_name, city, state, zipcode)
WHERE P.city = @city
  AND VR.vote_time BETWEEN @start_date AND @end_date
GROUP BY VR.acronym
HAVING COUNT(*) = (
    SELECT MAX(total)
    FROM (
        SELECT COUNT(*) AS total
        FROM VotingRegistry VR
        JOIN Places P USING (street_number, street_name, city, state, zipcode)
        WHERE P.city = @city
          AND VR.vote_time BETWEEN @start_date AND @end_date
        GROUP BY VR.acronym
    ) subquery
);
"""
cur.execute(query)
print(f"{'Voting Center':15} {'Total Registrations':20}")
print("=" * 40)
for row in cur:
    print(f"{row[0]:15} {row[1]:20}")
cur.close()


In [ ]:
print("Query 7")
cur = db_connection.cursor()
cur.execute("SET @state = 'State1';")
query = """
SELECT DISTINCT F.person_id, F.first_name, F.last_name
FROM Folks F
WHERE NOT EXISTS (
    SELECT VC.acronym
    FROM VotingCenter VC
    WHERE VC.state = @state
      AND NOT EXISTS (
          SELECT 1
          FROM VotingRegistry VR
          WHERE VR.person_id = F.person_id
            AND VR.acronym = VC.acronym
      )
);
"""
cur.execute(query)
print(f"{'Person ID':20} {'First Name':15} {'Last Name':15}")
print("=" * 50)
for row in cur:
    print(f"{row[0]:20} {row[1]:15} {row[2]:15}")
cur.close()



In [ ]:
print("Query 8")
cur = db_connection.cursor()
query = """
SELECT DISTINCT F.person_id, F.first_name, F.last_name, VR.acronym AS registered_center
FROM Folks F
JOIN VotingRegistry VR USING (person_id)
JOIN Places PR ON PR.street_number = F.residence_street_number 
              AND PR.street_name = F.residence_street_name 
              AND PR.city = F.residence_city 
              AND PR.state = F.residence_state 
              AND PR.zipcode = F.residence_zipcode
JOIN Places PV ON PV.street_number = VR.street_number 
              AND PV.street_name = VR.street_name 
              AND PV.city = VR.city 
              AND PV.state = VR.state 
              AND PV.zipcode = VR.zipcode
WHERE (
        POW(SUBSTRING_INDEX(PV.coordinates, ',', 1) - SUBSTRING_INDEX(PR.coordinates, ',', 1), 2) +
        POW(SUBSTRING_INDEX(PV.coordinates, ',', -1) - SUBSTRING_INDEX(PR.coordinates, ',', -1), 2)
      ) > (
        SELECT MIN(
            POW(SUBSTRING_INDEX(PC.coordinates, ',', 1) - SUBSTRING_INDEX(PR.coordinates, ',', 1), 2) +
            POW(SUBSTRING_INDEX(PC.coordinates, ',', -1) - SUBSTRING_INDEX(PR.coordinates, ',', -1), 2)
        )
        FROM VotingCenter VC
        JOIN Places PC ON PC.street_number = VC.street_number 
                       AND PC.street_name = VC.street_name 
                       AND PC.city = VC.city 
                       AND PC.state = VC.state 
                       AND PC.zipcode = VC.zipcode
    )
ORDER BY F.person_id, VR.acronym;
"""
cur.execute(query)
print(f"{'Person ID':20} {'First Name':15} {'Last Name':15} {'Registered Center':20}")
print("=" * 70)
for row in cur:
    print(f"{row[0]:20} {row[1]:15} {row[2]:15} {row[3]:20}")
cur.close()


In [ ]:
print("Query 9")
cur = db_connection.cursor()
cur.execute("SET @folk_id = '0000000000000001';") #Replace with the desired folk ID
cur.execute("SET @check_date = '2024-11-01';") #Replace with the desired date
query = """
SELECT VC.acronym AS closest_center
FROM VotingCenter VC
JOIN Places PC ON PC.street_number = VC.street_number 
               AND PC.street_name = VC.street_name 
               AND PC.city = VC.city 
               AND PC.state = VC.state 
               AND PC.zipcode = VC.zipcode
JOIN Folks F ON F.person_id = @folk_id
JOIN Places PR ON PR.street_number = F.residence_street_number 
               AND PR.street_name = F.residence_street_name 
               AND PR.city = F.residence_city 
               AND PR.state = F.residence_state 
               AND PR.zipcode = F.residence_zipcode
WHERE EXISTS (
    SELECT 1
    FROM Center_times CT
    WHERE CT.acronym = VC.acronym
      AND CT.start_time <= @check_date
      AND CT.end_time >= @check_date
)
ORDER BY (
    POW(SUBSTRING_INDEX(PC.coordinates, ',', 1) - SUBSTRING_INDEX(PR.coordinates, ',', 1), 2) +
    POW(SUBSTRING_INDEX(PC.coordinates, ',', -1) - SUBSTRING_INDEX(PR.coordinates, ',', -1), 2)
), VC.acronym
LIMIT 1;
"""
cur.execute(query)
result = cur.fetchone()
if result:
    print(f"Closest Voting Center: {result[0]}")
else:
    print("No valid voting center found.")
cur.close()


In [ ]:
print("Query 10")
cur = db_connection.cursor()
cur.execute("SET @ballot_id = 'B001';")
query = """
SELECT VC.acronym AS voting_center,
       SUM(CASE WHEN CV.choice = 'YES' THEN 1 ELSE 0 END) AS votes_yes,
       SUM(CASE WHEN CV.choice = 'NO' THEN 1 ELSE 0 END) AS votes_no,
       SUM(CASE WHEN CV.choice = 'ABSTAIN' THEN 1 ELSE 0 END) AS votes_abstain
FROM CastedVote CV
JOIN VotingCenter VC USING (acronym)
WHERE CV.ballot_id = @ballot_id
GROUP BY VC.acronym
ORDER BY VC.acronym;
"""
cur.execute(query)
print(f"{'Voting Center':15} {'Votes YES':10} {'Votes NO':10} {'Votes ABSTAIN':15}")
print("=" * 50)
for row in cur:
    print(f"{row[0]:15} {row[1]:10} {row[2]:10} {row[3]:15}")
cur.close()


In [ ]:
db_connection.close()